# keras embedding을 이용한 문서 분류
### 영화리뷰 데이터 대상

## train set의 크기가 더 큰 데이터 셋, 그리고 한글

In [1]:
import csv

text = []
y = []
with open('movie_data_new.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            text.append(row[0]) #영화 리뷰를 text 리스트에 추가
            y.append(row[2]) #영화이름을 text 리스트에 추가

In [2]:
print('Num of samples: {}'.format(len(text)))
print('Movie titles of reivews: {}'.format(set(y)))

Num of samples: 14967
Movie titles of reivews: {'라라랜드', '곤지암', '택시운전사', '범죄도시', '코코', '인피니티 워', '신과함께'}


In [3]:
label_dict = dict(zip(set(y),range(len(set(y)))))
y = [label_dict[c] for c in y]
print(len(y))
print(y[-10:])

14967
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [4]:
from keras.utils.np_utils import to_categorical
y = to_categorical(y)

AlreadyExistsError: Another metric with the same name already exists.

In [5]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()
#twitter_tag = Twitter()

In [6]:
print(twitter_tag.nouns(text[4]))

['내', '졸라']


## seqGenKor 에서 오류가 납니다 ..ㅠ 구글에 해당 모듈이 검색되지 않습니다

In [11]:
pip install seqGenKor

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement seqGenKor
ERROR: No matching distribution found for seqGenKor


In [13]:
from seqGenKor import SequenceGenKor

ModuleNotFoundError: No module named 'seqGenKor'

In [14]:
num_words = 10000
seq = SequenceGenKor(twitter_tag.nouns, num_words=num_words, min_token_len=1)
#seq = SequenceGenKor(twitter_tag.morphs, num_words=num_words, min_token_len=1)
seq.fit_on_texts(text)
X = seq.texts_to_sequences(text)
print('voca_size', len(seq.voca))

NameError: name 'SequenceGenKor' is not defined

In [15]:
print(X[:12])

NameError: name 'X' is not defined

In [16]:
text[:12]

['헐..다 죽었어....나중에 앤트맨 보다가도 깜놀...',
 '충격 결말',
 '응집력',
 '개연성은 무시해라 액션을 즐겨라 스타로드가 이끌어준다 각각의 영웅들을 즐겨라 그리고 단적인 신념이 얼마나 부질없는지 보셔라',
 '내가졸라이상하네',
 '대박',
 '정말 지루할틈없이 넘잘만들었다 역시 대단하다',
 '역시 어벤져스!!',
 '마지막에 누구한테 연락한거지? 궁금',
 '다음 편이 궁굼해지네요^^',
 '안잼있는사람 있음???????????',
 '타노스 개갞기']

In [17]:
from keras import preprocessing

maxlen = 20

X = preprocessing.sequence.pad_sequences(X, maxlen=maxlen, truncating='pre')
print(X[0])

AlreadyExistsError: Another metric with the same name already exists.

In [18]:
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
# sklearn의 train_test_split 함수는 먼저 data set을 shuffle하고 주어진 비율에 따라 train set과 test set을 나눠 줌
# 위에서는 reviews를 X_train과 X_test로 8:2의 비율로 나누고, categories를 y_train과 y_test로 나눔
# 이 때 X와 y의 순서는 동일하게 유지해서 각 입력값과 label이 정확하게 match되도록 함
# random_state는 shuffle에서의 seed 값으로, 지정한 경우 항상 동일한 결과로 shuffle이 됨

print('Train set count:', len(X_train))
print('Test set count:', len(X_test))
print('Test samples:', y_test[:20])

NameError: name 'X' is not defined

In [19]:
from keras.layers import SimpleRNN, LSTM
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(num_words, 32))
#model.add(SimpleRNN(32))
model.add(LSTM(32))
model.add(Dense(7, activation='softmax'))
model.summary()

AlreadyExistsError: Another metric with the same name already exists.

In [20]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train, 
                    epochs=30,
                    batch_size=256,
                    validation_split=0.2)

NameError: name 'model' is not defined

In [21]:
%matplotlib inline
import matplotlib.pyplot as plt

history_dict = history.history
acc = history_dict['acc']
val_acc = history_dict['val_acc']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [22]:
score = model.evaluate(X_test, y_test)
print(score)

NameError: name 'model' is not defined